# Identication of putative contaminant taxa

In [1]:
setwd("/mnt/c/Users/Cedric/Desktop/git_repos/blood_microbiome")
require(ggplot2)
require(ALDEx2)
require(tidyverse)
require(data.table)

Loading required package: ggplot2

Loading required package: ALDEx2

Loading required package: zCompositions

Loading required package: MASS

Loading required package: NADA

Loading required package: survival


Attaching package: ‘NADA’


The following object is masked from ‘package:stats’:

    cor


Loading required package: truncnorm

Loading required package: Rfast

Loading required package: Rcpp

Loading required package: RcppZiggurat

Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()     masks stats::filter()

#### Load abundance matrix

In [2]:
load_data <- function(file_path) {
    df <- fread(file_path) %>%
        separate(sample, into = c(NA, "npm_research_id"), sep = "\\.")
    return(df)
}

#### Load metadata

In [3]:
load_metadata <- function(file_path, df) {
    meta <- fread(file_path, na.strings=c("", NA)) %>%
        filter(npm_research_id %in% df$npm_research_id) %>%
        select(-removal_requested_by_supplier) %>%
        mutate(across(everything(), as.factor))

    return(meta)
}

#### Remove human and unclassified reads

In [4]:
remove_cols <- function(df, col_to_exclude) {
    filt_df <- df %>% select(-all_of(col_to_exclude))

    return(filt_df)
}

### Run ALDEX2

In [5]:
get_meta_cols <- function(meta) {
    meta_cols <- colnames(meta)[grep("kit|flow_cell|instrument_id|site_supplying", colnames(meta))]
    to_exclude <- c("library_prep_kit", "hiseq_xtm_flow_cell_v2_5_id")
    meta_cols <- meta_cols[!(meta_cols %in% to_exclude)]
    
    return(meta_cols)
}

In [6]:
get_meta_freqs <- function(meta, column) {
    tmp <- meta %>%
        group_by(get(column)) %>%
        summarise(n = n())
    
    return(tmp)
}

In [42]:
# run_aldex2 <- function(df, meta, meta_cols) {
#     result_df <- tibble()
    n_levels_list <- list()
    n_samples <- 10

#     for (column in meta_cols) {
        column <- "extraction_kit"
        column <- "instrument_id"
        print(str_glue("Running DESeq2 for {column} batch data"))
        # Remove levels with < n samples
        tmp <- get_meta_freqs(meta, column)

        # Vector of group levels to keep
        to_keep <- tmp[tmp$n >= n_samples, "get(column)"]$`get(column)`
        to_keep <- to_keep[to_keep != "Unknown"]
        n_levels_list[[column]] <- to_keep
        n_to_keep <- length(to_keep)
        n_original <- length(unique(tmp$`get(column)`))
        print(str_glue("No. of levels in {column} to keep: {n_to_keep} / {n_original}"))

        # Remove samples from meta
        meta_filt <- meta %>% 
            filter(get(column) %in% to_keep)
        
        # Retrieve filtered samples in abundance table
        df_filt <- df %>% 
            filter(npm_research_id %in% meta_filt$npm_research_id) %>%
            column_to_rownames("npm_research_id")
        
        # Convert abundance table to taxa x samples
        df_filt <- t(df_filt)

        nrow_to_keep <- ncol(df_filt)
        nrow_original <- nrow(df)
        print(str_glue("No. of samples to be analysed: {nrow_to_keep} / {nrow_original}"))
        
        x <- aldex.clr(df_filt, as.matrix(meta_filt)[, column], mc.samples = 1, denom = "all", verbose = T)

        morsel <- aldex.kw(x) %>% mutate(meta_col = column)

        result_df <- result_df %>% bind_rows(morsel)
result_df
#     }
    
#     return(list(result_df = result_df, n_levels_list = n_levels_list))
# }

Running DESeq2 for instrument_id batch data
No. of levels in instrument_id to keep: 5 / 6
No. of samples to be analysed: 384 / 663


operating in serial mode

removed rows with sums equal to zero

Warning message in aldex.clr.function(as.data.frame(reads), conds, mc.samples, denom, :
“values are unreliable when estimated with so few MC smps”
computing center with all features

data format is OK

dirichlet samples complete

transformation complete

operating in serial mode



,kw.ep,kw.eBH,glm.ep,glm.eBH,meta_col
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Bradyrhizobium sp. SK17...1,0.93293983,0.9924782,0.812216179,0.9812331,extraction_kit
Bradyrhizobium lablabi...2,0.50089979,0.9493643,0.699487275,0.9741794,extraction_kit
Bradyrhizobium ottawaense...3,0.22243025,0.9072522,0.230208187,0.9209002,extraction_kit
Bradyrhizobium sp. ORS 3257...4,0.51645760,0.9493643,0.690336458,0.9726414,extraction_kit
Bradyrhizobium oligotrophicum...5,0.01795602,0.7498830,0.012366417,0.7076544,extraction_kit
Bradyrhizobium sp. ORS 285...6,0.05270417,0.7506890,0.005027603,0.6658647,extraction_kit
Bradyrhizobium sp. BTAi1...7,0.16653904,0.8721596,0.195300847,0.9075179,extraction_kit
Bradyrhizobium icense...8,0.18976066,0.8816520,0.364665108,0.9489189,extraction_kit
Bradyrhizobium diazoefficiens...9,0.13864583,0.8637385,0.200435252,0.9075179,extraction_kit


#### Run decontamination on different subsets

In [26]:
# decontaminate <- function(n_subset) {
    n_subset <- 100
    # Load data
    df <- load_data(str_glue("data/temp_files_{n_subset}/07_abundance_matrix/abundance_matrix.subset_{n_subset}.S.tsv"))
    # Remove homo and unclassified
    col_to_exclude <- colnames(df)[grepl("Homo|unclassified", colnames(df))]
    df <- remove_cols(df, col_to_exclude)

    # Retain relevant metadata
    meta <- load_metadata("data/SG10K_Health_metadata.n10714.16March2021.parsed.csv", df)

    # Get metadata columns of interest
    meta_cols <- get_meta_cols(meta)
    
#    # Print batch levels and frequencies
#         print_group_freqs(meta, meta_cols)
    results <- run_aldex2(df, meta, meta_cols)
    results[['result_df']]

Running DESeq2 for site_supplying_sample batch data
No. of levels in site_supplying_sample to keep: 7 / 7
No. of samples to be analysed: 663 / 663


operating in serial mode

removed rows with sums equal to zero

Warning message in aldex.clr.function(as.data.frame(reads), conds, mc.samples, denom, :
“values are unreliable when estimated with so few MC smps”
computing center with all features

data format is OK

dirichlet samples complete

transformation complete

operating in serial mode



Running DESeq2 for extraction_kit batch data
No. of levels in extraction_kit to keep: 6 / 7
No. of samples to be analysed: 488 / 663


operating in serial mode

removed rows with sums equal to zero

Warning message in aldex.clr.function(as.data.frame(reads), conds, mc.samples, denom, :
“values are unreliable when estimated with so few MC smps”
computing center with all features

data format is OK

dirichlet samples complete

transformation complete

operating in serial mode



Running DESeq2 for instrument_id batch data
No. of levels in instrument_id to keep: 5 / 6
No. of samples to be analysed: 384 / 663


...

Running DESeq2 for hiseq_xtm_pe_cluster_kit_cbottm_v2__box_2_of_2__lot batch data
No. of levels in hiseq_xtm_pe_cluster_kit_cbottm_v2__box_2_of_2__lot to keep: 17 / 32
No. of samples to be analysed: 321 / 663


operating in serial mode

removed rows with sums equal to zero

Warning message in aldex.clr.function(as.data.frame(reads), conds, mc.samples, denom, :
“values are unreliable when estimated with so few MC smps”
computing center with all features

data format is OK

dirichlet samples complete

transformation complete

operating in serial mode



Running DESeq2 for hiseq_xtm_flow_cell_v2_5_lot batch data
No. of levels in hiseq_xtm_flow_cell_v2_5_lot to keep: 16 / 34
No. of samples to be analysed: 331 / 663


operating in serial mode

removed rows with sums equal to zero

Warning message in aldex.clr.function(as.data.frame(reads), conds, mc.samples, denom, :
“values are unreliable when estimated with so few MC smps”
computing center with all features

data format is OK

dirichlet samples complete

transformation complete

operating in serial mode



,kw.ep,kw.eBH,glm.ep,glm.eBH,meta_col
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Bradyrhizobium sp. SK17...1,0.64501504,0.9271805,0.487622969,0.9040059,site_supplying_sample
Bradyrhizobium lablabi...2,0.24662789,0.8219014,0.390124025,0.8965068,site_supplying_sample
Bradyrhizobium ottawaense...3,0.03345659,0.7135722,0.081553195,0.7512091,site_supplying_sample
Bradyrhizobium sp. ORS 3257...4,0.10934345,0.7750925,0.544290555,0.9131868,site_supplying_sample
Bradyrhizobium oligotrophicum...5,0.47319263,0.9099789,0.556889639,0.9134823,site_supplying_sample
Bradyrhizobium sp. ORS 285...6,0.15187100,0.7913174,0.139600138,0.8005682,site_supplying_sample
Bradyrhizobium sp. BTAi1...7,0.38754352,0.8833603,0.369518401,0.8959300,site_supplying_sample
Bradyrhizobium icense...8,0.25110525,0.8230853,0.395655315,0.8965068,site_supplying_sample
Bradyrhizobium diazoefficiens...9,0.22990389,0.8120158,0.120471991,0.8005682,site_supplying_sample


### Count number of covariates where taxa are flagged as non-diff abundant

In [37]:
result_df <- results[['result_df']]
result_df %>% 
    rownames_to_column("taxa") %>%
    mutate(diff_abn = ifelse(glm.ep < 0.05, F, T)) %>%
    group_by(taxa) %>%
    summarise(n = n_distinct(meta_col))
# %>%
#     pivot_wider(names_from = meta_col, values_from = diff_abn, id_cols = taxa)


taxa,n
<chr>,<int>
Acanthamoeba polyphaga mimivirus,1
Acanthocystis turfacea chlorella virus 1,1
Acaryochloris marina...13355,1
Acaryochloris marina...17065,1
Acaryochloris marina...1776,1
Acaryochloris marina...20808,1
Acaryochloris marina...24522,1
Acaryochloris marina...28214,1
Acaryochloris marina...5757,1


In [21]:
# fwrite(result_df, "results/decontamination/subset_100_aldex2.040821.csv")

### MAIN

In [ ]:
dadwwa

In [ ]:
# nc_list <- list()

# for (i in c(100)) {
#     nc_list[[str_glue("subset_{i}")]] <- decontaminate(i)
# }

# nc_list

### Sanity check

In [ ]:
# Reduce(intersect, nc_list)

In [ ]:
plot_sanity_plot <- function(X, Y, tax) {
    print(tax)
    temp_plt <- cbind(X, Y) %>%
        mutate(log_abundance = log(get(tax) + 0.0001)) %>%
        ggplot(aes(x = site_supplying_sample, y = get(tax), fill = site_supplying_sample)) + 
        geom_point() +
        geom_boxplot(alpha = 0.5) +
        labs(x = "Source cohort", y = paste(tax, "% abundance")) +
        theme(legend.position = "none",
          axis.text.x = element_text(angle = 45, hjust = 1),
          text = element_text(size = 15))
    return(temp_plt)
}

In [ ]:
# Load data
taxon <- "Cutibacterium acnes"
species_df <- load_data(str_glue("data/temp_files_{n_subset}/07_abundance_matrix/abundance_matrix.subset_{n_subset}.S.tsv"))

# Remove homo and unclassified
col_to_exclude <- colnames(species_df)[grepl("Homo|unclassified", colnames(species_df))]
species_df <- remove_cols(species_df, col_to_exclude)

# Retain relevant metadata
meta <- parse_metadata("data/SG10K_Health_metadata.n10714.16March2021.parsed.csv", species_df)

# Normalise to RA
normalise <- function(df) {
    RA_df <- t(apply(df, 1, function(x) {x / sum(x) * 100}))
    return(as_tibble(RA_df))
}

species_df_RA <- normalise(species_df)

plot_sanity_plot(species_df, meta, taxon)
print(str_glue("Max no. of reads = {max(species_df[, taxon])}"))
print(str_glue("Mean no. of reads = {mean(species_df[, taxon])}"))

In [ ]:
l1 <- tibble(l1)

# fwrite(l1, str_glue("/home/projects/14001280/PROJECTS/blood_microbiome/results/decontamination/decontaminated_list.n{n_subset}.csv"))